In [53]:
from fastai.vision import SegmentationItemList
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from file_cache import *
from fastai.vision import imagenet_stats


file1 = '/share/data2/body/brain/train/image/iNPH_MRI_PROCESSED/MRMR00093558-Yang JunSheng/3/MRMR00093558-Yang JunSheng_3.DCM_02_00000.png'
file2 = '/share/data2/body/brain/train/image/iNPH_MRI_PROCESSED/MRMR00010496-Ma ZhongYi/2/MRMR00010496-Ma ZhongYi_16.DCM_15_00000.png'
file3 = '/share/data2/body/brain/train/image/iNPH_MRI_PROCESSED/MRMR00075351-Zhang SiRu/4/MRMR00075351-Zhang SiRu_9.DCM_08_03524.png'
file4 = '/share/data2/body/brain/train/image/iNPH_MRI_PROCESSED/MRMR00093558-Yang JunSheng/3/MRMR00093558-Yang JunSheng_10.DCM_09_02900.png'
@lru_cache()
def get_df():
    img_file_list = glob('/share/data2/body/brain/train/image/*MRI*/**/*.*', recursive=True)

    df = pd.DataFrame({'img_file': [file4]*100})
    df['p_id_path'] = df.img_file.apply(lambda val: os.path.dirname(val))
    df['p_id'] = df['p_id_path'].rank(method='max').astype(int)
    df.sort_values('p_id')

    df['label_path'] = df.img_file.apply(lambda val: val.replace('image', 'label'))

    df['valid'] = df.p_id % 5 == 1
    
    

    df.valid.value_counts()
    return df



In [16]:
_default_anchors_setting = (
    dict(layer='p3', stride=32, size=48, scale=[2 ** (1. / 3.), 2 ** (2. / 3.)], aspect_ratio=[0.667, 1, 1.5]),
    dict(layer='p4', stride=64, size=96, scale=[2 ** (1. / 3.), 2 ** (2. / 3.)], aspect_ratio=[0.667, 1, 1.5]),
    dict(layer='p5', stride=128, size=192, scale=[1, 2 ** (1. / 3.), 2 ** (2. / 3.)], aspect_ratio=[0.667, 1, 1.5]),
)
_default_anchors_setting

({'layer': 'p3',
  'stride': 32,
  'size': 48,
  'scale': [1.2599210498948732, 1.5874010519681994],
  'aspect_ratio': [0.667, 1, 1.5]},
 {'layer': 'p4',
  'stride': 64,
  'size': 96,
  'scale': [1.2599210498948732, 1.5874010519681994],
  'aspect_ratio': [0.667, 1, 1.5]},
 {'layer': 'p5',
  'stride': 128,
  'size': 192,
  'scale': [1, 1.2599210498948732, 1.5874010519681994],
  'aspect_ratio': [0.667, 1, 1.5]})

In [13]:
np.zeros((0, 4), dtype=np.float32)

array([], shape=(0, 4), dtype=float32)

In [9]:
from file_cache import *
file1='/share/data2/body/brain/NPH_PROCESSED/sNPH_MRI_PROCESSED/MRMR00090067-Zhang JiLan'
file2='/share/data2/body/brain/NPH_PROCESSED/sNPH_MRI_PROCESSED/MRMR00143102-HUANGQINGHUA'
sorted(glob(f'{file2}/**/**.DCM', recursive=True), 
       key=lambda val: int(val.split('_')[-1].split('.')[0])
      )


['/share/data2/body/brain/NPH_PROCESSED/sNPH_MRI_PROCESSED/MRMR00143102-HUANGQINGHUA/MRMR00143102-HUANGQINGHUA_1.DCM',
 '/share/data2/body/brain/NPH_PROCESSED/sNPH_MRI_PROCESSED/MRMR00143102-HUANGQINGHUA/MRMR00143102-HUANGQINGHUA_2.DCM',
 '/share/data2/body/brain/NPH_PROCESSED/sNPH_MRI_PROCESSED/MRMR00143102-HUANGQINGHUA/MRMR00143102-HUANGQINGHUA_3.DCM',
 '/share/data2/body/brain/NPH_PROCESSED/sNPH_MRI_PROCESSED/MRMR00143102-HUANGQINGHUA/MRMR00143102-HUANGQINGHUA_4.DCM',
 '/share/data2/body/brain/NPH_PROCESSED/sNPH_MRI_PROCESSED/MRMR00143102-HUANGQINGHUA/MRMR00143102-HUANGQINGHUA_5.DCM',
 '/share/data2/body/brain/NPH_PROCESSED/sNPH_MRI_PROCESSED/MRMR00143102-HUANGQINGHUA/MRMR00143102-HUANGQINGHUA_6.DCM',
 '/share/data2/body/brain/NPH_PROCESSED/sNPH_MRI_PROCESSED/MRMR00143102-HUANGQINGHUA/MRMR00143102-HUANGQINGHUA_7.DCM',
 '/share/data2/body/brain/NPH_PROCESSED/sNPH_MRI_PROCESSED/MRMR00143102-HUANGQINGHUA/MRMR00143102-HUANGQINGHUA_8.DCM',
 '/share/data2/body/brain/NPH_PROCESSED/sNPH_MRI

In [ ]:
"""['/share/data2/body/brain/NPH_PROCESSED/sNPH_MRI_PROCESSED/MRMR00090067-Zhang JiLan/4/MRMR00090067-Zhang JiLan_12.DCM',"""


In [54]:
#ipdb.set_trace()
df = get_df()
src = (SegmentationItemList.from_df(df, path='/', cols='img_file')
       .split_from_df(col='valid')
       # .label_from_func(get_y_fn, classes=codes)
       .label_from_df(cols='label_path', classes=range(5))
       )
# .label_from_func(cols='label_path', classes=codes))

print(len(src.train), len(src.valid))
import ipdb
#
data = (src.transform(None, size=224 , tfm_y=True)
        .databunch(bs=8)
        
        )

#
print(data.train_dl.dl.worker_init_fn)
print(type(data.train_dl),data.train_dl.tfms)
print('==='*10)
data = data.normalize(imagenet_stats)
#print(type(data)) 
print(data.train_dl.dl.worker_init_fn)
print(type(data.train_dl),data.train_dl.tfms)

100 1
None
<class 'fastai.basic_data.DeviceDataLoader'> []
None
<class 'fastai.basic_data.DeviceDataLoader'> [functools.partial(<function _normalize_batch at 0x7f86551f30e0>, mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]), do_x=True, do_y=False)]


In [57]:
#ipdb.set_trace()
df = get_df()
df = df.iloc[1:]
src = (SegmentationItemList.from_df(df, path='/', cols='img_file')
       .split_from_df(col='valid')
       # .label_from_func(get_y_fn, classes=codes)
       .label_from_df(cols='label_path', classes=range(5))
       )
# .label_from_func(cols='label_path', classes=codes))

print(len(src.train), len(src.valid))
#import ipdb
#
print(type(src))
#ipdb.set_trace()
data = (src.transform(None, size=224 , tfm_y=True)  )
#ipdb.set_trace()
print(type(data))
data = data.databunch(bs=8)

print(type(data.train_ds))
for a, b in data.train_ds:
    print(a.shape, b.shape, a.data.sum(), b.data.sum(), np.unique(b.data))
    break

99 1
<class 'fastai.data_block.LabelLists'>
<class 'fastai.data_block.LabelLists'>
<class 'fastai.data_block.LabelList'>
torch.Size([3, 224, 224]) torch.Size([1, 224, 224]) tensor(21559.0781) tensor(3218) [0 1 2 3]


In [36]:
b fastai/data_block.py:516
b fastai/data_block.py:505
    
b fastai/data_block.py:726
    

[]

In [ ]:
batch_sampler',
 'batch_size',
 'collate_fn',
 'dataset',
 'drop_last',
 'init_kwargs',
 'multiprocessing_context',
 'num_workers',
 'pin_memory',
 'sampler',
 'timeout',
 'worker_init_fn'

In [6]:
dir(data.train_dl.dl)

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '_IterableDataset_len_called',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_index_sampler',
 'batch_sampler',
 'batch_size',
 'collate_fn',
 'dataset',
 'drop_last',
 'init_kwargs',
 'multiprocessing_context',
 'num_workers',
 'pin_memory',
 'sampler',
 'timeout',
 'worker_init_fn']

In [ ]:
b fastai/basic_data.py:117
b fastai/basic_data.py:47
b fastai/basic_data.py:64
b fastai/basic_data.py:81
b fastai/basic_data.py:46

b fastai/data_block.py:582
 
b fastai/vision/data.py:175
b fastai/vision/data.py:66
b fastai/vision/data.py:95
b fastai/torch_core.py:127

In [25]:
from fastai.basic_data import *

from dataset.ds_brain import *
# DataBunch().create(, DataSet_brain('valid'), DataSet_brain('valid'))

DataLoader(DataSet_brain('train'), 8, True, num_workers=8)

DataSet_brain {'self': <dataset.ds_brain.DataSet_brain object at 0x7f34b794e190>, 'ds_type': 'train', 'imgaug': True}
======== train
False    836
True     177
Name: valid, dtype: int64
> /home/aladdin1/anaconda3/lib/python3.7/site-packages/fastai/basic_data.py(16)intercept_args()
     15                         'drop_last': drop_last, 'timeout':timeout, 'worker_init_fn':worker_init_fn}
1--> 16     old_dl_init(self, dataset, **self.init_kwargs)
     17 

ipdb> c


In [10]:
!pip install inspect --user


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
ERROR: Could not find a version that satisfies the requirement inspect (from versions: none)
ERROR: No matching distribution found for inspect
You should consider upgrading via the '/home/aladdin1/anaconda3/bin/python -m pip install --upgrade pip' command.


In [5]:
data.train_dl.dl

In [7]:
import ipdb
ipdb.set_trace()


--Call--
> /home/aladdin1/anaconda3/lib/python3.7/site-packages/IPython/core/displayhook.py(252)__call__()
    251 
--> 252     def __call__(self, result=None):
    253         """Printing with history cache management.

ipdb> c
